##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [10]:
#%pip install pandas 
#%pip install matplotlib
#%pip install scikit-learn
import sklearn as sk
from sklearn import datasets
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [124]:
# Can have as many cells as you want for code
import pandas as pd
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

df = pd.read_parquet(filepath)

cols_df1 = df.columns[:8]  
cols_df2 = df.columns[8:18]  
cols_df3 = df.columns[18:24] 
cols_df4 = df.columns[24:34]  
cols_df5 = df.columns[34:41] 
cols_df6 = df.columns[41:158]  
cols_df7 = df.columns[158:303] #will be removed
cols_df8 = df.columns[303:] 

df1 = df[cols_df1]
df2 = df[cols_df2]
df3 = df[cols_df3]
df4 = df[cols_df4]
df5 = df[cols_df5]
df6 = df[cols_df6]
df7 = df[cols_df7]
df8 = df[cols_df8]

df1 = df1.drop(columns=df1.columns[:2])
df3 = df3[df3.columns[2:3]]
df3 = df3[df3.columns[2:3]]
df4 = df4.iloc[:, [0, 1, 2, 8, 9]]
df5 = df5.drop(columns = ['recency_cancel'])
df6 = df6.drop(columns=df6.columns[96:103])
df6 = df6.astype('float')


In [125]:
#combining columns in cat 6 based on ape, sumins and prempaid

ape_av = df6.columns[0:30].to_list() + df6.columns[89:96].to_list()
df6['av_ape'] = df6[ape_av].mean(axis=1)
df6 = df6.drop(columns=ape_av)

sumins_av = df6.columns[0:29].to_list() + df6.columns[59:66].to_list()
df6['av_sumins'] = df6[sumins_av].mean(axis=1)
df6 = df6.drop(columns=sumins_av)

prempaid_av = df6.columns[0:37]
df6['av_prempaid'] = df6[prempaid_av].mean(axis=1)
df6 = df6.drop(columns=prempaid_av)

df = pd.concat([df1, df2, df3, df4, df5, df6, df8], axis=1)
df.columns.get_loc("f_purchase_lh")
df = df.drop_duplicates()
df = df.drop('min_occ_date', axis=1)


### Dealing with missing data


In [126]:
# Detecting NaN and None

null = pd.isnull(df)
null.head()

# Filling null values

df.fillna(0)

#Count the total number of missing values
pd.isnull(df).sum().sum()

#dropped irrelevant/unhelpful rows
for x in df.index:
    if df.loc[x, "ctrycode_desc"] == "Not Applicable":
        df.drop(x, inplace=True)
        
for x in df.index:
    if df.loc[x, "annual_income_est"] == '':
        df.replace(to_replace = None, value = 0)
  



#knn for income
income = df['annual_income_est']
income.value_counts()
df["annual_income_est"][13:14]
from sklearn.impute import KNNImputer
#imputer_knn = KNNImputer(missing_values=None)
#imputer_knn.fit_transform(df["annual_income_est"])

#knn for recency lapse
imputer_knn = KNNImputer(missing_values=None)
#imputer_knn.fit_transform(df["recency_lapse"]) #tried to impute numerical values but knnimputer needs it to be a 2d array

csv_output = "./data/catB_train.csv" 
df.to_csv(csv_output, index = False)

#df = df.interpolate(method='pad')
df["f_purchase_lh"] = df["f_purchase_lh"].fillna(0)


## Feature Selection

In [127]:
#correlation between target and annual income
#df["av_ape"].corr(df["f_purchase_lh"]) #all the numeric comparisons gave nan output
#df = df.apply([pd.to_numeric]) #we tried to make all the categorical numeric but also did not work


df["av_ape"].corr(df["f_purchase_lh"])


0.04165033484771666

In [128]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#converting categorical to numerical
dummy_date = '1970-01-01'
df['cltdob_fix'] = df['cltdob_fix'].replace('None', dummy_date)
df['cltdob_fix'] = pd.to_datetime(df['cltdob_fix']).dt.year % 100

df['cltsex_fix'] = df['cltsex_fix'].map({'Male': 0, 'Female': 1})

df['hh_size_est'] = df['hh_size_est'].map({'1': 1, '2': 2, '3': 3, '4': 4, '>4': 5})

df['annual_income_est'] = df['cltsex_fix'].map({'annual_income_est_E.BELOW30K': 1, 'annual_income_est_D.30K-60K ': 2, 'annual_income_est_C.60K-100K': 3, 'annual_income_est_B.100K-200K': 4, 'annual_income_est_A.ABOVE200K': 5})



#one hot encoding
df = pd.get_dummies(df, columns=['ctrycode_desc', 'clttype', 'stat_flag'])


In [129]:
#decision tree classifier

from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier()
y = df["f_purchase_lh"]
X = df.drop(columns = ['f_purchase_lh'])
model.fit(X, y)

feature_importances = model.feature_importances_

feature_names = df.columns

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)
decision_tree_model.feature_importances_


ValueError: All arrays must be of the same length

SMOTE


In [23]:
#%pip install imblearn
from imblearn.over_sampling import SMOTE


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [25]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [26]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

[]


### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!